### Introduction 

### Setup

In [1]:
import pymongo
import pandas as pd
from pandas.io.json import json_normalize 
from pymongo import MongoClient
import numpy as np 

In [2]:
 #point the client at mongo URI 
client = MongoClient('mongodb://localhost:27017')
#select database
db = client['tipster']

In [3]:
# flatten data and put into dataframe 
meetings_collection = db.meetings
data=meetings_collection.find({},{"_id":0})
meeting_runners_df= pd.DataFrame(json_normalize(data,'runners',['meeting_url']))
meeting_runners_df['won'] = np.where(meeting_runners_df['place'] == 1.0, 1,0 )

In [69]:
form_collection = db.form
data = form_collection.find({},{"_id":0}) 
df = pd.DataFrame(json_normalize(data, 'form',['horse_name']))

### Data

### Features

which horses in a meeting are winners as of meeting date?  

has a horse in a meeting beaten another horse in the same meeting?

for a given pair of horses which one did better against a third horse?  
    how similar were the conditions? 

time since last race (fitness and value of form)

does the horse need to improve - is it a champion

will form improve?   
margin of victory - ( bigger implies form won't change )  (low fitness + ease => improving form) 

commentary:  
   activity during race  
   finish  
   speed 

is progressively better

above or below average time for course?

raised or lowered in class?

proven at distance 

proven for going

weight

draw - look up advantage of draw in course

type of race - handicap/flat/jumps

### Training

### Predict